# Homework 3 from Martin Gräf, Richard Baumann and Thomas Block

## Exercise 1
Derive corresponding expressions for O[φ] for  magnetization (per site) and energy (per site). 





## Exercise 2

Determine the equations of motion (EoMs) for this Hamiltonian by applying the
Hamilton equations


## Exercise 3

Code up the leapfrog algorithm to evolve (p0, φ0) → (pf , φf ). Verify the convergence claim H(pf , φf ) = H(p0, φ0) + O(2). You should find something similar to Figure 1
(especially for large Nmd)

## Exercise 5
With your working leapfrog integrator, code up the HMC algorithm for the long-range Ising model.

## Exercise 6
Setting h = (βh) = .5 and using some values of N ranging from 5 to 20, calculate the average energy per site and mean magnetization per site as a function of J = (βJ) ∈ [.2, 2]. Tune Nmd such that the acceptance rate is above 50%